<a href="https://colab.research.google.com/github/hirajya/Handling-Imbalance-dataset/blob/main/ANN_%7C_Handling_Imbalance_dataset_techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [4]:
path = '/content/drive/MyDrive/datasets/Churn_Modelling_Cleaned.csv'

df = pd.read_csv(path)

In [5]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,HasCrCard,IsActiveMember,EstimatedSalary,Exited,NumOfProducts_1,NumOfProducts_2,NumOfProducts_3,NumOfProducts_4
0,0.538,0,0.324324,0.2,0.000000,1,1,0.506735,1,1,0,0,0
1,0.516,0,0.310811,0.1,0.334031,0,1,0.562709,0,1,0,0,0
2,0.304,0,0.324324,0.8,0.636357,1,0,0.569654,1,0,0,1,0
3,0.698,0,0.283784,0.1,0.000000,0,0,0.469120,0,0,1,0,0
4,1.000,0,0.337838,0.2,0.500246,1,1,0.395400,0,1,0,0,0


In [7]:
len(df.columns)

13

In [18]:
def ANN(training_inputs, loss, weights):
  # training inputs = [X_train, X_test, y_train, y_test]
  model = keras.Sequential([
      keras.layers.Dense(10, input_shape=(12,), activation='relu'),
      keras.layers.Dense(5, activation='relu'),
      keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(
      optimizer='adam',
      loss='binary_crossentropy',
      metrics=['accuracy']
  )

  if weights == -1:
    model.fit(training_inputs[0], training_inputs[2], epochs=100)
  else:
    model.fit(training_inputs[0], training_inputs[2], epochs=100, class_weight = weights)

  print(model.evaluate(training_inputs[1], training_inputs[3]))

  y_preds = model.predict(training_inputs[1])
  y_pred = np.round(y_preds)

  print("Classification Report: \n", classification_report(training_inputs[3], y_pred))

  return y_preds

In [11]:
def train_input(X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

  return [X_train, X_test, y_train, y_test]


In [12]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,HasCrCard,IsActiveMember,EstimatedSalary,Exited,NumOfProducts_1,NumOfProducts_2,NumOfProducts_3,NumOfProducts_4
0,0.538,0,0.324324,0.2,0.000000,1,1,0.506735,1,1,0,0,0
1,0.516,0,0.310811,0.1,0.334031,0,1,0.562709,0,1,0,0,0
2,0.304,0,0.324324,0.8,0.636357,1,0,0.569654,1,0,0,1,0
3,0.698,0,0.283784,0.1,0.000000,0,0,0.469120,0,0,1,0,0
4,1.000,0,0.337838,0.2,0.500246,1,1,0.395400,0,1,0,0,0


In [20]:
df.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [21]:
# class count
count_class_0, count_class_1 = df.Exited.value_counts()

# divide by class
df_class_0 = df[df['Exited'] == 0]
df_class_1 = df[df['Exited'] == 1]

### Test; Control model

In [13]:
X = df.drop('Exited', axis=1)
y = df['Exited']

In [14]:
inputs = train_input(X, y)

In [19]:
y_pred_m0 = ANN(inputs, 'binary_crossentropy', -1)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5101 - accuracy: 0.7894
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4368 - accuracy: 0.8046
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4119 - accuracy: 0.8246
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3991 - accuracy: 0.8314
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3914 - accuracy: 0.8345
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3851 - accuracy: 0.8396
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3820 - accuracy: 0.8399
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3775 - accuracy: 0.8424
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3754 - accuracy: 0.8447
Epoch 10/100
250/250 [==============================] - 0s 1000us/step - loss: 0.3733 - acc

In [22]:
count_class_0, count_class_1

(7963, 2037)

In [23]:
df_class_0.shape

(7963, 13)

In [24]:
df_class_1.shape

(2037, 13)

### Method 1: Undersampling

In [26]:
df_class_0_under = df_class_0.sample(count_class_1)
df_class_0_under.shape

(2037, 13)

In [28]:
df_class_0_under[:5]

,CreditScore,Gender,Age,Tenure,Balance,HasCrCard,IsActiveMember,EstimatedSalary,Exited,NumOfProducts_1,NumOfProducts_2,NumOfProducts_3,NumOfProducts_4
4666,1.000,0,0.283784,0.5,0.456328,1,0,0.498437,0,1,0,0,0
5145,0.868,1,0.270270,0.1,0.552077,1,1,0.858868,0,1,0,0,0
7325,0.822,0,0.216216,0.2,0.000000,1,0,0.306228,0,0,1,0,0
1530,0.522,0,0.108108,0.2,0.428496,1,1,0.604008,0,0,1,0,0
4820,0.268,0,0.189189,0.3,0.000000,1,1,0.696964,0,0,1,0,0


In [29]:
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)
df_test_under.shape

(4074, 13)

In [30]:
df_test_under[:5]

,CreditScore,Gender,Age,Tenure,Balance,HasCrCard,IsActiveMember,EstimatedSalary,Exited,NumOfProducts_1,NumOfProducts_2,NumOfProducts_3,NumOfProducts_4
4666,1.000,0,0.283784,0.5,0.456328,1,0,0.498437,0,1,0,0,0
5145,0.868,1,0.270270,0.1,0.552077,1,1,0.858868,0,1,0,0,0
7325,0.822,0,0.216216,0.2,0.000000,1,0,0.306228,0,0,1,0,0
1530,0.522,0,0.108108,0.2,0.428496,1,1,0.604008,0,0,1,0,0
4820,0.268,0,0.189189,0.3,0.000000,1,1,0.696964,0,0,1,0,0


In [31]:
df_test_under.Exited.value_counts()

0    2037
1    2037
Name: Exited, dtype: int64

In [32]:
X = df_test_under.drop('Exited', axis=1)
y = df_test_under['Exited']

inputs_m1 = train_input(X, y)

In [33]:
y_pred_m1 = ANN(inputs_m1, 'binary_crossentropy', -1)

Epoch 1/100
102/102 [==============================] - 1s 2ms/step - loss: 0.6640 - accuracy: 0.6002
Epoch 2/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6101 - accuracy: 0.6892
Epoch 3/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5776 - accuracy: 0.7076
Epoch 4/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5580 - accuracy: 0.7128
Epoch 5/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5457 - accuracy: 0.7211
Epoch 6/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5373 - accuracy: 0.7257
Epoch 7/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5321 - accuracy: 0.7278
Epoch 8/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5278 - accuracy: 0.7297
Epoch 9/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5242 - accuracy: 0.7340
Epoch 10/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5216 - accura

### Method 2: Oversampling

In [34]:
count_class_0, count_class_1

(7963, 2037)

In [36]:
df_class_1_over = df_class_1.sample(count_class_0, replace=True) # gives duplication if value is less than perceived
df_class_1_over.shape

(7963, 13)

In [37]:
df_test_over = pd.concat([df_class_1_over, df_class_0], axis=0)
df_test_over.shape

(15926, 13)

In [38]:
df_test_over['Exited'].value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [40]:
X = df_test_over.drop('Exited', axis=1)
y = df_test_over['Exited']

inputs_m2 = train_input(X, y)

In [41]:
y_pred_m2 = ANN(inputs_m2, 'binary_crossentropy', -1)

Epoch 1/100
399/399 [==============================] - 2s 2ms/step - loss: 0.6094 - accuracy: 0.6680
Epoch 2/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5321 - accuracy: 0.7265
Epoch 3/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5160 - accuracy: 0.7392
Epoch 4/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5099 - accuracy: 0.7455
Epoch 5/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5062 - accuracy: 0.7480
Epoch 6/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5025 - accuracy: 0.7502
Epoch 7/100
399/399 [==============================] - 0s 1ms/step - loss: 0.5015 - accuracy: 0.7501
Epoch 8/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4995 - accuracy: 0.7505
Epoch 9/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4986 - accuracy: 0.7495
Epoch 10/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4975 - accura

### METHOD 3: Synthetic Minority Oversampling Technique (SMOTE)

In [42]:
X = df.drop('Exited', axis=1)
y = df['Exited']

In [43]:
y.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [44]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [46]:
inputs_m3 = train_input(X_sm, y_sm)
inputs_m3[0].shape

(12740, 12)

In [47]:
inputs_m3[1].shape

(3186, 12)

In [48]:
y_pred_m3 = ANN(inputs_m3, 'binary_crossentropy', -1)

Epoch 1/100
399/399 [==============================] - 2s 3ms/step - loss: 0.6864 - accuracy: 0.5502
Epoch 2/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5654 - accuracy: 0.7104
Epoch 3/100
399/399 [==============================] - 1s 2ms/step - loss: 0.5239 - accuracy: 0.7309
Epoch 4/100
399/399 [==============================] - 0s 1ms/step - loss: 0.5096 - accuracy: 0.7442
Epoch 5/100
399/399 [==============================] - 0s 1ms/step - loss: 0.5018 - accuracy: 0.7496
Epoch 6/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4973 - accuracy: 0.7529
Epoch 7/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4940 - accuracy: 0.7547
Epoch 8/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4911 - accuracy: 0.7561
Epoch 9/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4879 - accuracy: 0.7590
Epoch 10/100
399/399 [==============================] - 0s 1ms/step - loss: 0.4861 - accura

### Method 4: Use of Ensemble with undersampling

In [90]:
df.Exited.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [91]:
X = df.drop('Exited', axis=1)
y = df['Exited']

In [92]:
def get_train_batch(df_majority, df_minority, start, end):
  df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

  X_train = df_train.drop('Exited', axis=1)
  y_train = df_train.Exited

  return X_train, y_train

In [93]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)

In [94]:
def ANN(X_train, y_train, X_test, y_test, loss, weights):
  model = keras.Sequential([
      keras.layers.Dense(10, input_shape=(12,), activation='relu'),
      keras.layers.Dense(5, input_shape=(12,), activation='relu'),
      keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(
      optimizer='adam',
      loss='binary_crossentropy',
      metrics=['accuracy']
  )

  if weights == -1:
    model.fit(X_train, y_train, epochs=100)
  else:
    model.fit(X_train, y_train, epochs=100, class_weight = weights)

  print(model.evaluate(X_test, y_test))

  y_preds = model.predict(X_test)
  y_pred = np.round(y_preds)

  print("Classification Report: \n", classification_report(y_test, y_pred))

  return y_preds

In [95]:
y_train.value_counts()

0    6370
1    1630
Name: Exited, dtype: int64

In [96]:
6370/1630

3.9079754601226995

In [97]:
6370/3

2123.3333333333335

In [98]:
2100+2100+2170

6370

In [99]:
df2 = X_train.copy()
df2['Exited'] = y_train

df2_class0 = df2[df2.Exited==0]
df2_class1 = df2[df2.Exited==1]

df2_class0.shape, df2_class1.shape

((6370, 13), (1630, 13))

In [100]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 0, 2100)
y_pred_m4_1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
117/117 [==============================] - 1s 1ms/step - loss: 0.6727 - accuracy: 0.6097
Epoch 2/100
117/117 [==============================] - 0s 1ms/step - loss: 0.6209 - accuracy: 0.6826
Epoch 3/100
117/117 [==============================] - 0s 1ms/step - loss: 0.5855 - accuracy: 0.6962
Epoch 4/100
117/117 [==============================] - 0s 1ms/step - loss: 0.5669 - accuracy: 0.7008
Epoch 5/100
117/117 [==============================] - 0s 1ms/step - loss: 0.5541 - accuracy: 0.7113
Epoch 6/100
117/117 [==============================] - 0s 1ms/step - loss: 0.5446 - accuracy: 0.7169
Epoch 7/100
117/117 [==============================] - 0s 1ms/step - loss: 0.5376 - accuracy: 0.7249
Epoch 8/100
117/117 [==============================] - 0s 1ms/step - loss: 0.5328 - accuracy: 0.7244
Epoch 9/100
117/117 [==============================] - 0s 1ms/step - loss: 0.5275 - accuracy: 0.7263
Epoch 10/100
117/117 [==============================] - 0s 1ms/step - loss: 0.5243 - accura

In [101]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 2100, 4200)
y_pred_m4_2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
117/117 [==============================] - 1s 1ms/step - loss: 0.6740 - accuracy: 0.5568
Epoch 2/100
117/117 [==============================] - 0s 1ms/step - loss: 0.6231 - accuracy: 0.6252
Epoch 3/100
117/117 [==============================] - 0s 2ms/step - loss: 0.5974 - accuracy: 0.6836
Epoch 4/100
117/117 [==============================] - 0s 1ms/step - loss: 0.5776 - accuracy: 0.7038
Epoch 5/100
117/117 [==============================] - 0s 2ms/step - loss: 0.5618 - accuracy: 0.7080
Epoch 6/100
117/117 [==============================] - 0s 1ms/step - loss: 0.5482 - accuracy: 0.7126
Epoch 7/100
117/117 [==============================] - 0s 1ms/step - loss: 0.5378 - accuracy: 0.7182
Epoch 8/100
117/117 [==============================] - 0s 2ms/step - loss: 0.5303 - accuracy: 0.7231
Epoch 9/100
117/117 [==============================] - 0s 2ms/step - loss: 0.5214 - accuracy: 0.7263
Epoch 10/100
117/117 [==============================] - 0s 3ms/step - loss: 0.5182 - accura

In [102]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 4200, 6370)
y_pred_m4_3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy', -1)

Epoch 1/100
119/119 [==============================] - 1s 1ms/step - loss: 0.6819 - accuracy: 0.6371
Epoch 2/100
119/119 [==============================] - 0s 1ms/step - loss: 0.6223 - accuracy: 0.6874
Epoch 3/100
119/119 [==============================] - 0s 1ms/step - loss: 0.5738 - accuracy: 0.7034
Epoch 4/100
119/119 [==============================] - 0s 1ms/step - loss: 0.5547 - accuracy: 0.7113
Epoch 5/100
119/119 [==============================] - 0s 1ms/step - loss: 0.5416 - accuracy: 0.7184
Epoch 6/100
119/119 [==============================] - 0s 1ms/step - loss: 0.5339 - accuracy: 0.7279
Epoch 7/100
119/119 [==============================] - 0s 1ms/step - loss: 0.5271 - accuracy: 0.7292
Epoch 8/100
119/119 [==============================] - 0s 1ms/step - loss: 0.5215 - accuracy: 0.7345
Epoch 9/100
119/119 [==============================] - 0s 1ms/step - loss: 0.5173 - accuracy: 0.7371
Epoch 10/100
119/119 [==============================] - 0s 1ms/step - loss: 0.5143 - accura

In [103]:
print(len(y_pred_m4_1))
print(len(y_pred_m4_2))
print(len(y_pred_m4_3))


2000
2000
2000


In [105]:
y_pred_final = y_pred_m4_1.copy()

for i in range(len(y_pred_m4_1)):
  n_ones = y_pred_m4_1[i] + y_pred_m4_2[i] + y_pred_m4_3[i]

  if n_ones > 1:
    y_pred_final[i] = 1
  else:
    y_pred_final[i] = 0

In [106]:
y_pred_final

array([[1.],
       [0.],
       [1.],
       ...,
       [1.],
       [0.],
       [0.]], dtype=float32)

In [107]:
print(classification_report(y_test, y_pred_final))

              precision    recall  f1-score   support

           0       0.94      0.65      0.77      1593
           1       0.38      0.85      0.53       407

    accuracy                           0.69      2000
   macro avg       0.66      0.75      0.65      2000
weighted avg       0.83      0.69      0.72      2000

